# Introduction

After creating the Tapis pod for a Neo4j database, we need to add data to it. It is very easy to load in data to a csv 

In [2]:
import py2neo
import pandas
from py2neo import Graph, Node, Relationship
from py2neo import GraphService
import time
from getpass import getpass
from tapipy.tapis import Tapis

def show(res):
    try:
        pp.pprint(res.json())
    except:
        pp.pprint(res.text)

In [3]:
start = time.time()

# Base URL for Tapis
base_url = "https://icicle.develop.tapis.io"
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    t
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

6.911173582077026
base_url: https://icicle.develop.tapis.io
serv_url: https://icicle.develop.tapis.io/v3


In [4]:
t.pods.get_pods()

[
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The pod for the REHS final KG
 environment_variables: 
 
 pod_id: finalkg
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-07-29 20:20:36+00:00
 status_requested: ON
 update_ts: None
 url: finalkg.pods.icicle.develop.tapis.io,
 
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The second pod for the REHS final project.
 environment_variables: 
 
 pod_id: finalkg2
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-08-02 09:48:46+00:00
 status_requested: ON
 update_ts: None
 url: finalkg2.pods.icicle.develop.tapis.io]

In [5]:
# See pod ID using above jupyter cell
pod_id = str(input("Enter a pod ID. ")).lower()
username, password = t.pods.get_pod_credentials(pod_id=pod_id).user_username, t.pods.get_pod_credentials(pod_id=pod_id).user_password

In [6]:
# Connection to neo4j database
graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.develop.tapis.io:443", auth=(username, password), secure=True, verify=True)

# How does it work?
We first call LOAD CSV WITH HEADERS and then give it a url which contains the CSV file to load.

Merge associates a set of nodes with labels to a column or columns of data. For example, in the below code, nodes with label PLATFORM are created which contain data from the row with header Platform. Similarly, nodes with label GAME contain the specified data from the rows.

We can also create relationships upon load. The below example simply creates a relationship from every game to the platform it is associated with.

In [ ]:
# Loading CSV Example
script = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/vgsales.csv" AS row
MERGE (n:PLATFORM {{id:row.Platform}})
MERGE (n1:GAME {{id:row.Name, rank:row.Rank, year:row.Year, genre:row.Genre, 
publisher:row.Publisher, NA_Sales:row.NA_Sales, EU_SALES:row.EU_Sales,JP_Sales:row.JP_Sales, 
OTHER_Sales:row.Other_Sales}})
MERGE (n1)-[:PART_OF]->(n)
"""
graph.run(script).to_data_frame()